In [ ]:
import os
import sys
import json
import time
import pyvisa
import numpy as np
from scipy import signal
from pathlib import Path
from dataclasses import dataclass
from typing import List, Optional, Tuple, Dict
import requests

print("All dependency libraries imported successfully!")

In [ ]:
class BVCTools:
    """Signal analyzer for Jupyter environment"""
    
    def __init__(self, ):
        pass
        
    def sync(self, device_ip: str, module_name: str, board_name: str):

        _,_,_ = self.link_BVC(device_ip)

        print()
       
        # Send RF auto-sync command
        self.scpi_query(f':exe {module_name},0')

        self.scpi_query(f':SYS:REFerence:SAT:STEP1 {module_name}')
        self.scpi_query(f':SYS:REFerence:SAT:STEP2 {module_name}')
        self.scpi_query(f':SAT:MULTisync:SAT:STEP3 {board_name}')
        self.scpi_query(f':SYS:REFerence:SAT:STEP4 {module_name}')
        self.scpi_query(f':SAT:MULTisync:SAT:STEP5 {board_name}')
        self.scpi_query(f':SYS:REFerence:SAT:STEP6 {module_name}')
        self.scpi_query(f':SAT:MULTisync:SAT:STEP7 {board_name}')

        self.scpi_query(f':exe {module_name},14')

    def link_BVC(self, device_ip: str):
        visa_resource = f'TCPIP::{device_ip}::5555::SOCKET'
        
        rm = pyvisa.ResourceManager('@py')

        session = rm.open_resource(visa_resource)  # open resource
        session.write_termination = '\n'
        session.read_termination = '\n'
        session.timeout = 50000  # timeout unit: ms
        self.scpi_query = session.query
        self.scpi_write = session.write
        self.scpi_read = session.read
        return self.scpi_query, self.scpi_write, self.scpi_read

    # FFT function
    def plot_fft(self, image_num, signal, t: np.ndarray, fs: float=20e6, fc: float=1e6, phase: float=0):
        plt.figure(figsize=(16, 8))

        # Calculate spectrum (-fs/2, fs/2)
        
        # Direct FFT calculation with default points
        fft = np.fft.fft(signal) # Generate spectrum [0, fs) by default
        freq  = np.fft.fftfreq(len(t), d=1/fs) # Default frequency axis [-fs/2, fs/2)
        freq_shifted = np.fft.fftshift(freq)  # Symmetric frequency format
        
        fft_shifted = np.fft.fftshift(fft) # Shift spectrum to [-fs/2, fs/2)
        fft_freq_fs = np.linspace(0, fs, len(fft), endpoint=False) # Generate frequency axis [0, fs)
        
        plt.subplot(2, 1, 1)
        plt.stem(fft_freq_fs, fft, basefmt=" ")
        plt.title(f'No.{image_num} Data Signal FFT (0, fs)')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Magnitude')
        
        plt.grid()
        
        plt.figure(figsize=(16, 8))
        plt.subplot(2, 1, 2)
        plt.stem(freq_shifted, fft_shifted, basefmt=" ")
        plt.title(f'No.{image_num} Data Signal FFT (~fs/2, fs/2)')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Magnitude')
        
        plt.grid()
        plt.show()
        
        print(f"signal fft max: {max(fft)}")
        print(f"signal fft shifted max: {max(fft)}")

print("Jupyter signal generator class definition completed!")